In [1]:
import hail as hl
hl.init()

Running on Apache Spark version 2.4.5
SparkUI available at http://hail-py-m.us-central1-b.c.cncd-cncd.internal:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.61-3c86d3ba497a
LOGGING: writing to /hail-20201217-0658-0.2.61-3c86d3ba497a.log


In [7]:
data = hl.import_plink(bed='gs://ucsf-dev/output/PROMIS_GWAS1_chr15.bed',
                      bim='gs://ucsf-dev/output/PROMIS_GWAS1_chr15.bim',
                      fam='gs://ucsf-dev/output/PROMIS_GWAS1_chr15.fam',
                      reference_genome='GRCh37')
data.rows().show()

2020-12-17 08:07:31 Hail: INFO: Found 9226 samples in fam file.
2020-12-17 08:07:31 Hail: INFO: Found 292639 variants in bim file.


,,,
locus,alleles,rsid,cm_position
locus<GRCh37>,array<str>,str,float64
15:20000538,"[""C"",""T""]","""rs201108160:20000538:C:T""",0.00e+00
15:20000638,"[""C"",""T""]","""rs28859984:20000638:T:C""",0.00e+00
15:20001056,"[""G"",""C""]","""rs200712166:20001056:G:C""",0.00e+00
15:20001075,"[""G"",""T""]","""15:20001075:G:T""",0.00e+00
15:20001079,"[""A"",""C""]","""rs61997852:20001079:A:C""",0.00e+00
15:20001087,"[""A"",""C""]","""rs61997853:20001087:A:C""",0.00e+00
15:20001090,"[""T"",""A""]","""rs368406015:20001090:T:A""",0.00e+00
15:20001226,"[""C"",""T""]","""rs28896870""",0.00e+00


In [8]:
a = data.filter_rows(data.rsid.contains('rs5029904' or 'rs3825807' or 'rs7178051'))
a.rows().show()

,,,
locus,alleles,rsid,cm_position
locus<GRCh37>,array<str>,str,float64
15:79152422,"[""C"",""G""]","""rs5029904:79152422:C:G""",0.00e+00


In [10]:
# recode1 = {f"{i}":f"chr{i}" for i in (list(range(1, 23)) + ['X', 'Y',])}
# recode2 = { "MT":"chrM" }
# def merge_two_dicts(x, y):
#     """Given two dictionaries, merge them into a new dict as a shallow copy."""
#     z = x.copy()
#     z.update(y)
#     return z
# recode = merge_two_dicts(recode1, recode2)

for dataset,size in zip(['gs://pgr-wes-30k-mt/Complete_Annotations/Canonical/30K_Chr15_VEP_DB_C.mt',
             'gs://pgr-wgs-8k/VEP_VCF/pgr8k_chr15_VEP.mt',
             'gs://pgr-wgs-2k/VEP_VCF/Chr15_2k_wgs.mt'],
                ['30k', '8k', '2k']):

# try:
#     data = hl.import_plink(bed='gs://ucsf-dev/output/PROMIS_GWAS1_chr15.bed',
#                       bim='gs://ucsf-dev/output/PROMIS_GWAS1_chr15.bim',
#                       fam='gs://ucsf-dev/output/PROMIS_GWAS1_chr15.fam',
#                       reference_genome='GRCh37')
    
    ds = hl.read_matrix_table(dataset)

#     ds = hl.import_plink(bed="gs://ucsf-dev/cncd_ucsf_columbia/pVCF/clean/chr1.bed",
#                          bim="gs://ucsf-dev/cncd_ucsf_columbia/pVCF/clean/chr1.bim",
#                          fam="gs://ucsf-dev/cncd_ucsf_columbia/pVCF/clean/chr1.fam")

    rs_ids = ['rs3825807', 'rs7178051', 'rs5029904']

    data = ds.filter_rows( (ds.rsid.contains(rs_ids[0] or rs_ids[1] or rs_ids[2])) ) #|(ds.rsid=='rs7178051')|(ds.rsid=='rs5029904')
    a = data.count()
    if a[0] == 0:
        print('No SNP Found')
        data.rows().show()
    else:

        print('Annotate Homs-0 Alleles')
        ds_result1 = data.annotate_rows(homs0 = hl.agg.filter(data.GT.is_hom_ref(),hl.agg.collect(data.s)))
        ds_result4 = ds_result1.annotate_rows(n_homs0 = hl.agg.count_where(ds_result1.GT.is_hom_ref()))
        print('Annotate Hets-1 Alleles')
        ds_result1 = ds_result4.annotate_rows(hets1 = hl.agg.filter(ds_result4.GT.is_het(),hl.agg.collect(ds_result4.s)))
        ds_result4 = ds_result1.annotate_rows(n_hets1 = hl.agg.count_where(ds_result1.GT.is_het()))
        print('Annotate Homs-2 Alleles')
        ds_result3 = ds_result4.annotate_rows(homs2 = hl.agg.filter(ds_result4.GT.is_hom_var(),hl.agg.collect(ds_result4.s)))
        ds_result5 = ds_result3.annotate_rows(n_homs2 = hl.agg.count_where(ds_result3.GT.is_hom_var()))
        ds_result5.rows().show()
        
#         for gt in ['homs0', 'hets1', 'homs2']:
        tab0 = ds_result5.rows().explode('homs0')
#         tab0.show()
        !rm *.xlsx
        ds_result6 = tab0.key_by()

        print('Converting To Pandas')
        finData_All = ds_result6.select(
            locus=ds_result6.locus,
            alleles=ds_result6.alleles,
            gene=ds_result6.rsid,
            homs0=ds_result6.homs0,
        #     n_homs0=ds_result6.n_homs0,
        #     hets21=ds_result6.hets1,
        #     n_hets1=ds_result6.n_hets1,
        #     homs2=ds_result6.homs2,
        #     n_homs2=ds_result6.n_homs2
        ).to_pandas()      

        finData_All.to_excel(str(size)+"-0.xlsx")
        !gsutil cp *.xlsx gs://test1haseeb/ucsf/PROMIS_GWAS1/
            
        tab0 = ds_result5.rows().explode('hets1')
#         tab0.show()
        !rm *.xlsx
        ds_result6 = tab0.key_by()

        print('Converting To Pandas')
        finData_All = ds_result6.select(
            locus=ds_result6.locus,
            alleles=ds_result6.alleles,
            gene=ds_result6.rsid,
#             homs0=ds_result6.homs0,
        #     n_homs0=ds_result6.n_homs0,
            hets21=ds_result6.hets1,
        #     n_hets1=ds_result6.n_hets1,
        #     homs2=ds_result6.homs2,
        #     n_homs2=ds_result6.n_homs2
        ).to_pandas()      

        finData_All.to_excel(str(size)+"-1.xlsx")
        !gsutil cp *.xlsx gs://test1haseeb/ucsf/PROMIS_GWAS1/
            
        tab0 = ds_result5.rows().explode('homs2')
#         tab0.show()
        !rm *.xlsx
        ds_result6 = tab0.key_by()

        print('Converting To Pandas')
        finData_All = ds_result6.select(
            locus=ds_result6.locus,
            alleles=ds_result6.alleles,
            gene=ds_result6.rsid,
#             homs0=ds_result6.homs0,
        #     n_homs0=ds_result6.n_homs0,
        #     hets21=ds_result6.hets1,
        #     n_hets1=ds_result6.n_hets1,
            homs2=ds_result6.homs2,
        #     n_homs2=ds_result6.n_homs2
        ).to_pandas()      

        finData_All.to_excel(str(size)+"-2.xlsx")
        !gsutil cp *.xlsx gs://test1haseeb/ucsf/
            
# except Exception as e:
#     print(e)

Annotate Homs-0 Alleles
Annotate Hets-1 Alleles
Annotate Homs-2 Alleles


+---------------+------------+-------------+----------+----------+
| locus         | alleles    | rsid        |     qual | filters  |
+---------------+------------+-------------+----------+----------+
| locus<GRCh37> | array<str> | str         |  float64 | set<str> |
+---------------+------------+-------------+----------+----------+
| 15:79089111   | ["A","G"]  | "rs3825807" | 4.00e+07 | {}       |
+---------------+------------+-------------+----------+----------+

+--------------+----------------+---------+-------------------+
| info.AC      | info.AF        | info.AN | info.BaseQRankSum |
+--------------+----------------+---------+-------------------+
| array<int32> | array<float64> |   int32 |           float64 |
+--------------+----------------+---------+-------------------+
| [58803]      | [3.76e-01]     |  156588 |          1.93e+00 |
+--------------+----------------+---------+-------------------+

+----------------------+---------+---------+---------+----------+----------+
| info.ClippingRankSum | info.DB | info.DP | info.DS | info.END |  info.FS |
+----------------------+---------+---------+---------+----------+----------+
|              float64 |    bool |   int32 |    bool |    int32 |  float64 |
+----------------------+---------+---------+---------+----------+----------+
|             0.00e+00 |    true | 3344591 |   false |       NA | 0.00e+00 |
+----------------------+---------+---------+---------+----------+----------+

+---------------------+----------------------+--------------+----------------+
| info.HaplotypeScore | info.InbreedingCoeff | info.MLEAC   | info.MLEAF     |
+---------------------+----------------------+--------------+----------------+
|             float64 |              float64 | array<int32> | array<float64> |
+---------------------+----------------------+--------------+----------------+
|                  NA |             5.26e-02 | [58805]      | [3.76e-01]     |
+---------------------+----------------------+--------------+----------------+

+----------+----------------+--------------------------+
|  info.MQ | info.MQRankSum | info.NEGATIVE_TRAIN_SITE |
+----------+----------------+--------------------------+
|  float64 |        float64 |                     bool |
+----------+----------------+--------------------------+
| 5.93e+01 |       1.36e-01 |                    false |
+----------+----------------+--------------------------+

+--------------------------+----------+---------------------+----------+
| info.POSITIVE_TRAIN_SITE |  info.QD | info.ReadPosRankSum | info.SOR |
+--------------------------+----------+---------------------+----------+
|                     bool |  float64 |             float64 |  float64 |
+--------------------------+----------+---------------------+----------+
|                     true | 1.87e+01 |            3.96e-01 | 7.03e-01 |
+--------------------------+----------+---------------------+----------+

+-------------+--------------+-------------------+-------------------+
| info.VQSLOD | info.culprit | vep.assembly_name | vep.allele_string |
+-------------+--------------+-------------------+-------------------+
|     float64 | str          | str               | str               |
+-------------+--------------+-------------------+-------------------+
|    5.63e+00 | "FS"         | "GRCh37"          | "A/G"             |
+-------------+--------------+-------------------+-------------------+

+---------------+
| vep.ancestral |
+---------------+
| str           |
+---------------+
| NA            |
+---------------+

+------------------------------------------------------------------------------+
| vep.colocated_variants                                                       |
+------------------------------------------------------------------------------+
| array<struct{aa_allele: str, aa_maf: float64, afr_allele: str, afr_maf: f... |
+------------------------------------------------------------------------------+
| [("G",3.24e-01,"G",3.54e-01,"A/G","G",1.11e

Converting To Pandas
Copying file://30k-0.xlsx [Content-Type=application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]...
/ [1 files][365.7 KiB/365.7 KiB]                                                
Operation completed over 1 objects/365.7 KiB.                                    
Converting To Pandas
Copying file://30k-1.xlsx [Content-Type=application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]...
/ [1 files][379.6 KiB/379.6 KiB]                                                
Operation completed over 1 objects/379.6 KiB.                                    
Converting To Pandas
Copying file://30k-2.xlsx [Content-Type=application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]...
/ [1 files][121.8 KiB/121.8 KiB]                                                
Operation completed over 1 objects/121.8 KiB.                                    
Annotate Homs-0 Alleles
Annotate Hets-1 Alleles
Annotate Homs-2 Alleles


+----------------+------------+-------------+----------+----------+------------+
| locus          | alleles    | rsid        |     qual | filters  | info.AVGDP |
+----------------+------------+-------------+----------+----------+------------+
| locus<GRCh38>  | array<str> | str         |  float64 | set<str> |    float64 |
+----------------+------------+-------------+----------+----------+------------+
| chr15:78796769 | ["A","G"]  | "rs3825807" | 2.55e+02 | {}       |   3.72e+01 |
+----------------+------------+-------------+----------+----------+------------+

+--------------+---------+-------------+----------------+-------------+
| info.AC      | info.AN | info.FIBC_P | info.HWE_SLP_P | info.FIBC_I |
+--------------+---------+-------------+----------------+-------------+
| array<int32> |   int32 |     float64 |        float64 |     float64 |
+--------------+---------+-------------+----------------+-------------+
| [5393]       |   14962 |    8.01e-02 |       1.00e+02 |    4.53e-03 |
+--------------+---------+-------------+----------------+-------------+

+----------------+-------------+-------------+
| info.HWE_SLP_I | info.MAX_IF | info.MIN_IF |
+----------------+-------------+-------------+
|        float64 |     float64 |     float64 |
+----------------+-------------+-------------+
|       1.42e+00 |    4.61e-01 |    5.39e-02 |
+----------------+-------------+-------------+

+---------------------------------------------------------+----------+
| info.BETA_IF                                            | info.ABE |
+---------------------------------------------------------+----------+
| array<float64>                                          |  float64 |
+---------------------------------------------------------+----------+
| [2.79e-01,2.35e+00,3.92e+00,9.06e-01,9.57e-02,1.00e+00] | 4.94e-01 |
+---------------------------------------------------------+----------+

+-----------+--------------+----------+----------+-----------+-----------+
|  info.ABZ | info.NS_NREF | info.BQZ | info.MQZ |  info.CYZ |  info.STZ |
+-----------+--------------+----------+----------+-----------+-----------+
|   float64 |        int32 |  float64 |  float64 |   float64 |   float64 |
+-----------+--------------+----------+----------+-----------+-----------+
| -2.39e+01 |           NA | 1.42e+02 | 1.15e-01 | -5.27e+00 | -8.27e+00 |
+-----------+--------------+----------+----------+-----------+-----------+

+----------+-----------+----------+----------+----------------+-----------+
| info.NMZ |  info.IOR | info.NM0 | info.NM1 | info.FLT20     | info.LLK0 |
+----------+-----------+----------+----------+----------------+-----------+
|  float64 |   float64 |  float64 |  float64 | array<float64> |   float64 |
+----------+-----------+----------+----------+----------------+-----------+
| 9.79e+01 | -1.46e-01 | 4.36e-01 | 3.42e-01 | NA             | -2.22e+05 |
+----------+-----------+----------+----------+----------------+-----------+

+------------+----------+
| info.GWAS  | info.SVM |
+------------+----------+
| array<str> |  float64 |
+------------+----------+
| ["PASS"]   | 2.21e+00 |
+------------+----------+

+------------------------------------------------------------------------------+
| info.ANN                                                                     |
+------------------------------------------------------------------------------+
| array<str>                                                                   |
+------------------------------------------------------------------------------+
| ["G|missense_variant|MODERATE|ADAMTS7|ENSG00000136378|transcript|ENST0000... |
+------------------------------------------------------------------------------+

+-------------+-----------------+-----------------------+
| info.FMIS10 | info.MAX_HM3_R2 | info.LOG_HM3_BUDDY_03 |
+-------------+-----------------+-----------------------+
|     float64 |         float64 |               float64 |
+-------------+-----------------+-----------------------+

Converting To Pandas
Copying file://8k-0.xlsx [Content-Type=application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]...
/ [1 files][ 93.4 KiB/ 93.4 KiB]                                                
Operation completed over 1 objects/93.4 KiB.                                     
Converting To Pandas
Copying file://8k-1.xlsx [Content-Type=application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]...
/ [1 files][ 99.8 KiB/ 99.8 KiB]                                                
Operation completed over 1 objects/99.8 KiB.                                     
Converting To Pandas
Copying file://8k-2.xlsx [Content-Type=application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]...
/ [1 files][ 34.1 KiB/ 34.1 KiB]                                                
Operation completed over 1 objects/34.1 KiB.                                     
Annotate Homs-0 Alleles
Annotate Hets-1 Alleles
Annotate Homs-2 Alleles


+----------------+------------+-------------+----------+----------+
| locus          | alleles    | rsid        |     qual | filters  |
+----------------+------------+-------------+----------+----------+
| locus<GRCh38>  | array<str> | str         |  float64 | set<str> |
+----------------+------------+-------------+----------+----------+
| chr15:78796769 | ["A","G"]  | "rs3825807" | 1.82e+06 | {}       |
+----------------+------------+-------------+----------+----------+

+--------------+----------------+---------+-------------------+
| info.AC      | info.AF        | info.AN | info.BaseQRankSum |
+--------------+----------------+---------+-------------------+
| array<int32> | array<float64> |   int32 |           float64 |
+--------------+----------------+---------+-------------------+
| NA           | NA             |      NA |                NA |
+--------------+----------------+---------+-------------------+

+----------------------+---------+---------+---------+----------+---------+
| info.ClippingRankSum | info.DB | info.DP | info.DS | info.END | info.FS |
+----------------------+---------+---------+---------+----------+---------+
|              float64 |    bool |   int32 |    bool |    int32 | float64 |
+----------------------+---------+---------+---------+----------+---------+
|                   NA |   false |      NA |   false |       NA |      NA |
+----------------------+---------+---------+---------+----------+---------+

+---------------------+----------------------+--------------+----------------+
| info.HaplotypeScore | info.InbreedingCoeff | info.MLEAC   | info.MLEAF     |
+---------------------+----------------------+--------------+----------------+
|             float64 |              float64 | array<int32> | array<float64> |
+---------------------+----------------------+--------------+----------------+
|                  NA |                   NA | NA           | NA             |
+---------------------+----------------------+--------------+----------------+

+---------+----------+----------------+--------------------------+
| info.MQ | info.MQ0 | info.MQRankSum | info.NEGATIVE_TRAIN_SITE |
+---------+----------+----------------+--------------------------+
| float64 |    int32 |        float64 |                     bool |
+---------+----------+----------------+--------------------------+
|      NA |       NA |             NA |                    false |
+---------+----------+----------------+--------------------------+

+--------------------------+---------+---------------------+----------+
| info.POSITIVE_TRAIN_SITE | info.QD | info.ReadPosRankSum | info.SOR |
+--------------------------+---------+---------------------+----------+
|                     bool | float64 |             float64 |  float64 |
+--------------------------+---------+---------------------+----------+
|                    false |      NA |                  NA |       NA |
+--------------------------+---------+---------------------+----------+

+-------------+--------------+---------+-----------+--------------------------+
| info.VQSLOD | info.culprit | a_index | was_split | variant_qc.dp_stats.mean |
+-------------+--------------+---------+-----------+--------------------------+
|     float64 | str          |   int32 |      bool |                  float64 |
+-------------+--------------+---------+-----------+--------------------------+
|          NA | NA           |       1 |     false |                 2.36e+01 |
+-------------+--------------+---------+-----------+--------------------------+

+---------------------------+-------------------------+
| variant_qc.dp_stats.stdev | variant_qc.dp_stats.min |
+---------------------------+-------------------------+
|                   float64 |                 float64 |
+---------------------------+-------------------------+
|                  7.68e+00 |                2.00e+00 |
+---------------------------+-------------------------+

+-------------------------+--------------------------+
| v

rm: cannot remove '*.xlsx': No such file or directory
Converting To Pandas
Copying file://2k-0.xlsx [Content-Type=application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]...
/ [1 files][ 25.8 KiB/ 25.8 KiB]                                                
Operation completed over 1 objects/25.8 KiB.                                     
Converting To Pandas
Copying file://2k-1.xlsx [Content-Type=application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]...
/ [1 files][ 27.4 KiB/ 27.4 KiB]                                                
Operation completed over 1 objects/27.4 KiB.                                     
Converting To Pandas
Copying file://2k-2.xlsx [Content-Type=application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]...
/ [1 files][ 11.7 KiB/ 11.7 KiB]                                                
Operation completed over 1 objects/11.7 KiB.                                     
